In [186]:
# First we import numpy and pandas.  The following is just convention for people working with packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

## First let's get some data

We will use the kaggle dataset for 80 cereals and their nutrition. The file is included with this repository.

### Importing

Pandas has some impressive tools to import data.  One of the most common ones is `read_csv`

In [187]:
pd.read_csv?

In [188]:
df = pd.read_csv('cereal.csv', sep=',', header=0)

## Now lets take a quick look and see if it imported correctly

### `head()` and `tail()`
The `head()` method is really nice for a fast look at your data frame.  If you mess up the header row it will tell you really quickly. By default both `head()` and `tail()` return 5 rows by default

### `info()`
`info()` lists the columns, tell you how many non-null values arein there, and the numpy type

### `describe()`
Gives a really nice stats layout of the data

In [189]:
df.head()

,name,mfr,type,calories,protein,fat,sodium,fiber,carbo,sugars,potass,vitamins,shelf,weight,cups,rating
0,100% Bran,N,C,70,4,1,130,10.0,5.0,6,280,25,3,1.0,0.33,68.402973
1,100% Natural Bran,Q,C,120,3,5,15,2.0,8.0,8,135,0,3,1.0,1.00,33.983679
2,All-Bran,K,C,70,4,1,260,9.0,7.0,5,320,25,3,1.0,0.33,59.425505
3,All-Bran with Extra Fiber,K,C,50,4,0,140,14.0,8.0,0,330,25,3,1.0,0.50,93.704912
4,Almond Delight,R,C,110,2,2,200,1.0,14.0,8,-1,25,3,1.0,0.75,34.384843


In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

## Data Frame and Series

### Data Frames

You can think of a DataFrame as a spreadsheet.  It has columns and rows containing the data.  Each row has an index and each column has some type of name.

### Series

Series are a data type that is simply a series of values either a single row or a single column

In [ ]:
df[0:5] #same as head

## Getting at your data

There are several ways to access data.  

In [ ]:
# Get a column by name
df['name']

In [ ]:
# Get a column by . access
df.name

In [ ]:
# If you slice a dataframe it returns the columns
df[0:4]

In [ ]:
# You can also skip
df[0:8:2]

### `iloc`

In [ ]:
# iloc finds by position
df.iloc[0:3]

In [ ]:
# iloc finds by position
df.iloc[0:3, 0:3]

### `loc` is by label

In [ ]:
df.loc[0:3]

In [ ]:
df.loc[0:3, ['name', 'mfr', 'type']]

#### NOTE:  `ix` is deprecated so don't use

### Filtering Items


Most common way is using masks.  This is not as crazy as it sounds and often don't even see them.

In [ ]:
# Notice how it returns ALL rows but with a True or False
df.mfr == 'N'

In [ ]:
# Let's drop that into the dataframe selection
df[df.mfr == 'N']

In [ ]:
# Want more specifics
df[(df.mfr == 'N') & (df.calories < 100)]

### Only want certain columns?

Just pass a list in the same way you filtered

In [ ]:
df[['name', 'mfr', 'type']].head()

### String searching

In [ ]:
df[df.name.str.contains('Frosted', case=False)]

## Sorting time

In [ ]:
df.sort_values(by='name')

In [ ]:
df.sort_index(ascending=False)

## Lets change the data

## Quick Reindexing

In [ ]:
# Quick copy
df_reindexed = df.copy()

In [ ]:
# Set the index
df_reindexed.index = df_reindexed['name']
df_reindexed.head()

In [ ]:
# Drop the name column since it is the index now
df_reindexed = df_reindexed.drop('name', axis=1)
df_reindexed.head()

In [ ]:
df_reindexed.head()

In [ ]:
df_reindexed.loc['All-Bran':'Almond Delight']

### Let's fix the -1 in the columns.

An easier way to do this would be on import

In [ ]:
columns = ["calories", "protein", "fat", "sodium", "fiber", "carbo", "sugars", "potass", "vitamins", "shelf", "weight"]

In [ ]:
df_reindexed[columns] = df_reindexed[columns].replace(-1, np.NaN)

In [ ]:
df_reindexed.head()

### Replace a specific item

In [ ]:
df_reindexed.at['Almond Delight', 'potass'] = 300

In [ ]:
df_reindexed.head()

## Stats Watch your numbers

Remember `df` has all original values.  `df_reindexed` we replaced -1 with `np.NaN`

In [ ]:
df.sugars.mean()

In [ ]:
df_reindexed.sugars.mean()

## Graphs are good

In [ ]:
df.sugars.plot(kind="hist")

In [ ]:
df.calories.plot(kind="hist")

In [ ]:
plt.scatter(df.calories, df.sugars)

## Let's Fill out mfr

The letters aren't great so lets also add a full manufacturer name

In [ ]:
# First the dictionary.  Already here for you
mfrs = {'A': 'American Home Food Products', 
        'G': 'General Mills', 
        'K': 'Kelloggs',
        'N': 'Nabisco',
        'P': 'Post',
        'Q': 'Quaker Oats',
        'R': 'Ralston Purina'}

In [ ]:
# Now lets use replace get a new column
df.mfr.replace(mfrs)

In [ ]:
df['mfr_full'] = df.mfr.replace(mfrs)

In [ ]:
df.head()

## A simple groupby

In [ ]:
df.groupby(by=['mfr_full']).count()

## Apply

Sometimes you need to do calculations to transform the data in a column to something else.  Pandas allows for a function call with the values to transform them.

In [ ]:
# For example potass and sodium are measured in mg not g like the other measurements.  
# Lets change sodium to grams

df.sodium.apply(lambda x: x/1000)

In [ ]:
df['sodium_g'] = df.sodium.apply(lambda x: x/1000)

In [ ]:
df.head()

### More complex apply method

We can do more complex things to entire rows.  Let's standardize the values so EVERYTHING is based on 1 cup serving size.

In [ ]:
df_std = df.copy()

In [ ]:
def standardize(row):
    # first get multiplier for the cup
    multi = 1/row.cups
    
    # columns we care about
    columns = ['calories', 'protein', 'fat', 'sodium', 'fiber', 'carbo', 'sugars', 'potass', 'vitamins', 'cups']
    row[columns] = row[columns] * multi
    return row
    
df_std = df_std.apply(standardize, axis=1)

In [ ]:
df_std

In [ ]:
plt.scatter(df_std.calories, df_std.sugars)

In [ ]:
df_std.sort_values('calories', ascending=False)[0:5]

In [ ]:
plt.scatter(df_std[df_std.calories < 300].calories, df_std[df_std.calories < 300].sugars)

In [ ]:
df_std[df_std.calories < 300]

## Merge

Merging is similar to SQL with specifying which join to use

In [ ]:
pd.merge(df, df_std, on="name")